In [28]:
import numpy as np
import pandas as pd

In [623]:
import inflect

p = inflect.engine()
numbers_in_words = np.array([p.number_to_words(i) for i in range(-2, 13)])
size = numbers_in_words.shape[0]

print(numbers_in_words)

['minus two' 'minus one' 'zero' 'one' 'two' 'three' 'four' 'five' 'six'
 'seven' 'eight' 'nine' 'ten' 'eleven' 'twelve']


In [624]:
print(size)

15


In [641]:
def encode(data):
  x_one_hot_encode = list()
  x_vectors = list()
  for i in range(data.shape[0]):
    array = np.zeros((data.shape[0], 1))
    array[i] = 1
    x_one_hot_encode.append(array)
    x_vectors.append(np.random.uniform(-.3, .3, (3, 1)))

  dataset = pd.DataFrame()
  dataset["name"] = data
  dataset["encoding"] = x_one_hot_encode
  dataset["vector"] = x_vectors
  return dataset

In [642]:
dataset = encode(numbers_in_words)
dataset

,name,encoding,vector
0,minus two,"[[1.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[-0.22937643195657598], [-0.07772392642098205..."
1,minus one,"[[0.0], [1.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.1055277593859657], [-0.15233415004616752],..."
2,zero,"[[0.0], [0.0], [1.0], [0.0], [0.0], [0.0], [0....","[[-0.23944332182969197], [-0.2540282428402471]..."
3,one,"[[0.0], [0.0], [0.0], [1.0], [0.0], [0.0], [0....","[[-0.2441660517272983], [-0.11655617148161337]..."
4,two,"[[0.0], [0.0], [0.0], [0.0], [1.0], [0.0], [0....","[[0.008843530306965053], [-0.07759034478113302..."
5,three,"[[0.0], [0.0], [0.0], [0.0], [0.0], [1.0], [0....","[[-0.023430695870813645], [0.11381677920331057..."
6,four,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1....","[[0.2325283411640015], [0.07011401047138321], ..."
7,five,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.08987418523836688], [0.23395098093715755],..."
8,six,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[-0.1465066811097934], [-0.15304093440143068]..."
9,seven,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.08825779454630611], [0.09356702346339468],..."


In [627]:
print(dataset.loc[0, "encoding"].shape)

(15, 1)


In [677]:
mapping = dict()
for i in range(dataset.loc[0, "encoding"].shape[0]):
  mapping[tuple(dataset.loc[i, "encoding"].flatten())] = dataset.loc[i, "name"]
  mapping[dataset.loc[i, "name"]] = i

In [678]:
def training_set(dataset):
  x_train = np.empty((dataset.loc[0, "encoding"].shape[0]-4), dtype=np.ndarray)
  y_train = np.empty((dataset.loc[0, "encoding"].shape[0]-4), dtype=np.ndarray)
  for i in range(x_train.shape[0]):
    index = i+2
    x_train[i] = dataset.loc[index, "encoding"]
    y_train[i] = dataset.loc[index-2, "encoding"] + dataset.loc[index-1, "encoding"] +dataset.loc[index+1, "encoding"] +dataset.loc[index+2, "encoding"]

  return x_train, y_train

In [630]:
print(x_train[0].shape)

(205, 1)


In [631]:
x_train, y_train = training_set(dataset)
print(x_train[0], "\n")
print(y_train[0])

[[0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] 

[[1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [632]:
class Layer:

  def __init__(self, m, n):
    self.parameters = np.random.uniform(-.3, .3, (m,n))
    self.gradient = np.zeros((m,n))
    self.mults = np.zeros((m, m))
    self.size = (m, n)

  def __mul__(self, other):
    self.mults = other
    return np.dot(self.parameters, other)

  def __rmul__(self, other):
    return self * other

  def zero_gradient(self):
    self.gradient = np.zeros(self.size)

  def __repr__(self):
    return f"{self.parameters}"

In [723]:
class Skip_gram_model:

  def __init__(self, dimensions, x_train, y_train, mapping, dataset):
    self.size = x_train.shape[0] + 4
    self.dim = dimensions
    self.hidden_layer = Layer(dimensions, self.size)
    self.output_layer = Layer(self.size, dimensions)
    self.softmax = np.array([0.0])
    self.x_train = x_train
    self.y_train = y_train
    self.mapping = mapping
    self.dataset = dataset

  def feed_forward(self, input):
    a1 = self.hidden_layer * input
    a2 = self.output_layer * a1
    exp_output = np.exp(a2)
    exp_sum = np.sum(exp_output)
    soft_max = exp_output/exp_sum
    self.softmax = soft_max
    return soft_max

  def predict(self, input):
    index = np.argmax(self.feed_forward(input).flatten())
    arr = np.zeros((self.size), dtype=int)
    arr[index] = 1
    return self.mapping[tuple(arr)]

  def gradient_descent(self, loss):
    self.output_layer.gradient += np.dot(loss, self.output_layer.mults.transpose())
    self.hidden_layer.gradient += np.dot(np.dot(self.output_layer.parameters.transpose(), loss), self.hidden_layer.mults.transpose())

  def zero_gradient(self):
    self.hidden_layer.zero_gradient()
    self.output_layer.zero_gradient()

  def back_propigate(self, a):
    self.hidden_layer.parameters -= a * self.hidden_layer.gradient
    self.output_layer.parameters -= a * self.output_layer.gradient

  def epoch(self, a):
    self.zero_gradient()
    cost = 0.0
    for i in range(self.x_train.shape[0]):
      pred = self.feed_forward(self.x_train[i])
      self.gradient_descent(pred - self.y_train[i])
      loss = self.y_train[i] * np.log(pred)
      cost += loss
    self.back_propigate(a)
    return -np.sum(cost)/self.size

  def train(self, a, epochs):
    initial_cost = self.epoch(a)
    cost = initial_cost
    for i in range(epochs):
      c = self.epoch(a)
      print(f"i = {i}, cost = {c}")
      if (c > cost):
        break
      cost = c

    for i in range(self.dataset.loc[0, "encoding"].shape[0]):
      self.dataset.loc[i, "vector"] = np.dot(self.hidden_layer.parameters, self.dataset.loc[i, "encoding"])
    return initial_cost - cost

  def add(self, a, b):
    a = np.array(self.dataset.loc[self.mapping[a], "vector"])
    b = np.array(self.dataset.loc[self.mapping[b], "vector"])
    sum = a + b
    # print(a.shape)
    # print(b.shape)

    cosine_similarity = np.zeros(self.dataset.loc[0, "encoding"].shape[0])
    for i in range(self.dataset.loc[0, "encoding"].shape[0]):
      c = np.array(self.dataset.loc[i, "vector"])
      # print(c.shape)
      dot_product = np.dot(sum, c)
      #cosine_similarity[i] = dot_product
      cosine_similarity[i] = dot_product/(np.linalg.norm(sum) * np.linalg.norm(c))
    return self.dataset.loc[np.argmax(cosine_similarity), "name"]



In [734]:
model = Skip_gram_model(3, x_train, y_train, mapping, dataset)

In [740]:
print(model.train(4e-3, 500))

i = 0, cost = 6.750567086172714
0.0


In [729]:
numbers_in_words

array(['minus two', 'minus one', 'zero', 'one', 'two', 'three', 'four',
       'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve'],
      dtype='<U9')

In [730]:
model.add("five", "five")

'minus two'

In [686]:
type(dataset.loc[0, "vector"])

list

In [679]:
dataset

,name,encoding,vector
0,minus two,"[[1.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[0.038460872353882736, 0.2653112709617233, 0.1..."
1,minus one,"[[0.0], [1.0], [0.0], [0.0], [0.0], [0.0], [0....","[0.1720441602882254, -0.030229605986560992, -0..."
2,zero,"[[0.0], [0.0], [1.0], [0.0], [0.0], [0.0], [0....","[-0.13271942589835445, 0.027950651351906334, -..."
3,one,"[[0.0], [0.0], [0.0], [1.0], [0.0], [0.0], [0....","[0.12536086228239685, 0.21037334562805687, -0...."
4,two,"[[0.0], [0.0], [0.0], [0.0], [1.0], [0.0], [0....","[0.5082985825776087, 0.5303946209800603, 0.144..."
5,three,"[[0.0], [0.0], [0.0], [0.0], [0.0], [1.0], [0....","[-0.2242237572017159, 0.5149637641002356, 0.39..."
6,four,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1....","[-0.10963961421295232, -0.4113393776696443, -0..."
7,five,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[0.4664249075105761, 0.10299922381403481, -0.1..."
8,six,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[0.653622517062769, 0.6085388932569824, -0.044..."
9,seven,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[-0.1189287772452913, -0.31085494314666856, -0..."


In [402]:
# class Neuron:

#   def __init__(self, number_of_weights):
#     self.parameters = np.random.uniform(-.3, .3, (number_of_weights, ))
#     self.gradients = np.zeros((number_of_weights, ))
#     self.previous_gradients = np.zeros((number_of_weights, ))
#     self.mults = np.zeros((number_of_weights, ))
#     self.size = number_of_weights

#   def feed_forward(self, input):
#     self.mults = input
#     dot_p = np.dot(self.parameters, input)
#     return dot_p

#   def gradient(self, neuron_gradient):
#     self.gradients = np.add(self.gradients, self.mults * neuron_gradient)
#     return self.parameters * neuron_gradient

#   def zero_gradient(self):
#     self.gradients = np.zeros((self.size, ))

#   def back_propigate(self, a, g):
#     dt = self.gradients * -a
#     self.parameters += dt + g * self.previous_gradients
#     self.previous_gradients = dt


#   def __repr__(self):
#     return f"Neuron {self.parameters}"



# class Layer:

#   def __init__(self, weights_per_neuron, number_of_neurons):
#     self.layer = np.array([Neuron(weights_per_neuron) for _ in range(number_of_neurons)])
#     self.activation_gradients = np.zeros((number_of_neurons, ))
#     self.size = number_of_neurons

#   def feed_forward(self, input):
#     x_values = input
#     activations = np.zeros((self.size, ))
#     for i in range(self.size):
#       neuron_output = self.layer[i].feed_forward(x_values)
#       activations[i] = neuron_output
#       self.activation_gradients[i] = 1.0
#     return activations

#   def gradient(self, previous_layer_gradients):
#     neuron_gradients = np.multiply(self.activation_gradients, previous_layer_gradients)
#     layer_gradients = np.zeros((self.layer[0].size, ))
#     for i in range(self.size):
#       layer_gradients = np.add(layer_gradients, self.layer[i].gradient(neuron_gradients[i]))
#     return layer_gradients

#   def zero_gradient(self):
#     for i in range(self.size):
#       self.layer[i].zero_gradient()

#   def back_propigate(self, a, g):
#     for i in range(self.size):
#       self.layer[i].back_propigate(a, g)

#   def __repr__(self):
#     s = ""
#     for i in range(self.size):
#       s += str(self.layer[i])+"\n"
#     return s

# class NeuralNetwork:

#   def __init__(self, x_train, y_train):
#     self.size = 2
#     self.network = np.array([Layer(x_train.shape[0]+4, 3), Layer(3, x_train.shape[0]+4)])
#     self.x_train = x_train
#     self.y_train = y_train

#   def feed_forward(self, input):
#     previous_layer_activations = input
#     for i in range(self.size):
#       temp = self.network[i].feed_forward(previous_layer_activations)
#       previous_layer_activations = np.copy(temp)

#     return previous_layer_activations

#   def predict(self, x_data):
#     predictions = np.zeros((len(x_data), ))
#     for i in range(len(x_data)):
#       predictions[i] = self.feed_forward(x_data[i])
#     return predictions

#   def gradient(self, dE_dOutput):
#     last_layer_gradients = np.array([dE_dOutput for _ in range(self.network[self.size-1].size + 1)])
#     previous_gradients = self.network[self.size-1].gradient(last_layer_gradients)
#     if (self.size<2):
#       return

#     for i in range(self.size-2, -1, -1):
#       temp = self.network[i].gradient(previous_gradients)
#       previous_gradients = temp

#   def zero_gradient(self):
#     for i in range(self.size):
#       self.network[i].zero_gradient()

#   def back_propigate(self, a, g):
#     for i in range(self.size):
#       self.network[i].back_propigate(a, g)

#   def epoch(self, a, g):
#     self.zero_gradient()
#     cost = 0.0
#     for i in range(self.x_train.shape[0]):
#       pred = self.feed_forward(self.x_train[i])
#       self.gradient(pred - self.y_train[i])
#       loss = self.y_train[i] * np.log(pred)
#       cost += loss
#     self.back_propigate(a, g)
#     return -np.sum(cost)/self.size

#   def train(self, a, g):
#     initial_cost = self.epoch(a, g)
#     cost = initial_cost
#     for i in range(1):
#       c = epoch(a, g)
#       if (c > cost):
#         break
#       cost = c

#     return f"{initial_cost}, {cost}, {initial_cost-cost}"

#   def __repr__(self):
#     s = ""
#     for i in range(self.size):
#       s += str(self.network[i])+"\n"
#     return s

In [439]:
# x_train_2 = np.empty((x_train.shape), dtype=np.ndarray)
# y_train_2 = np.empty((x_train.shape), dtype=np.ndarray)
# for i in range(x_train.shape[0]):
#   x_train_2[i] = x_train[i].reshape(1, 205)[0]
#   y_train_2[i] = y_train[i].reshape(1, 205)[0]

In [440]:
# print(x_train_2[0], "\n")
# print(y_train_2[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 

[1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0

In [441]:
# model = NeuralNetwork(x_train_2, y_train_2)

In [442]:
# model.train(4e-3, 0.0)

ValueError: ignored